In [ ]:
!git clone https://github.com/pizadi/ISIC2018-training.git
from os import chdir
chdir('./ISIC2018-training')

In [ ]:
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Validation_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Validation_GroundTruth.zip
# !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip

In [ ]:
!unzip ./ISIC2018_Task1-2_Training_Input.zip
!rm ./ISIC2018_Task1-2_Training_Input.zip
!unzip ./ISIC2018_Task1_Training_GroundTruth.zip
!rm ./ISIC2018_Task1_Training_GroundTruth.zip
!unzip ./ISIC2018_Task1-2_Validation_Input.zip
!rm ./ISIC2018_Task1-2_Validation_Input.zip
!unzip ./ISIC2018_Task1_Validation_GroundTruth.zip
!rm ./ISIC2018_Task1_Validation_GroundTruth.zip
!mkdir ./Preproc

In [ ]:
!pip install torchinfo

In [ ]:
import os
import torch
from torch import nn as nn
# import cv2 as cv
import torchvision
from tqdm import tqdm
from matplotlib import pyplot as plt
import torchinfo
from DoubleUNet import DoubleUNet

In [ ]:
TRAIN_INPUT_DIR = './ISIC2018_Task1-2_Training_Input/'
TRAIN_GT_DIR = './ISIC2018_Task1_Training_GroundTruth/'

VAL_INPUT_DIR = './ISIC2018_Task1-2_Validation_Input/'
VAL_GT_DIR = './ISIC2018_Task1_Validation_GroundTruth/'

BATCH_SIZE = 4
LEARNING_RATE = 1e-4
EPOCHS = 100
IM_H, IM_W = 512, 768

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")

In [ ]:
training_input_files = os.listdir(TRAIN_INPUT_DIR)
for filename in training_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    training_input_files.remove(filename)
n_train = len(training_input_files)

val_input_files = os.listdir(VAL_INPUT_DIR)
for filename in val_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    val_input_files.remove(filename)
n_val = len(val_input_files)
n_train -= n_train % BATCH_SIZE
n_val -= n_val % BATCH_SIZE
print(n_train, n_val)

In [ ]:
train_X = torch.zeros((BATCH_SIZE, 3, IM_H, 384)).to(DEVICE)
train_y = torch.zeros((BATCH_SIZE, 1, IM_H, 384)).to(DEVICE)
nb = n_train//BATCH_SIZE
with tqdm(total=n_train) as pbar:
    for f in range(nb):
        l, r = f*BATCH_SIZE, (f+1)*BATCH_SIZE
        filename = None
        for i in range(l, r):
            filename = training_input_files[i]
            in_image = torchvision.io.read_image(TRAIN_INPUT_DIR + filename).to(DEVICE)
            gt_image = torchvision.io.read_image(TRAIN_GT_DIR + filename[:-4] + "_segmentation.png").to(DEVICE)
            train_X[i-l, :, :, :] = torchvision.transforms.functional.resize(in_image, [IM_H, IM_W])
            train_y[i-l, :, :, :] = torchvision.transforms.functional.resize(gt_image, [IM_H, IM_W])
            in_image, gt_image = None, None
            torch.cuda.empty_cache()
            pbar.update(1)

        train_y = (train_y//128)
        torch.save(train_X, f"./Preproc/train_X_{f}.torch")
        torch.save(train_y, f"./Preproc/train_y_{f}.torch")
train_X, train_y = None, None
torch.cuda.empty_cache()
   
val_X = torch.zeros((BATCH_SIZE, 3, IM_H, IM_W)).to(DEVICE)
val_y = torch.zeros((BATCH_SIZE, 1, IM_H, IM_W)).to(DEVICE)
nb = n_val//BATCH_SIZE
with tqdm(total=n_val) as pbar:
    for f in range(nb):
        l, r = f*BATCH_SIZE, (f+1)*BATCH_SIZE
        filename = None
        for i in range(l, r):
            filename = val_input_files[i]
            in_image = torchvision.io.read_image(VAL_INPUT_DIR + filename).to(DEVICE)
            gt_image = torchvision.io.read_image(VAL_GT_DIR + filename[:-4] + "_segmentation.png").to(DEVICE)
            val_X[i-l, :, :, :] = torchvision.transforms.functional.resize(in_image, [IM_H, IM_W])
            val_y[i-l, :, :, :] = torchvision.transforms.functional.resize(gt_image, [IM_H, IM_W])
            in_image, gt_image = None, None
            torch.cuda.empty_cache()
            pbar.update(1)

        val_y = (val_y//128)
        torch.save(val_X, f"./Preproc/val_X_{f}.torch")
        torch.save(val_y, f"./Preproc/val_y_{f}.torch")
val_X, val_y = None, None
torch.cuda.empty_cache()
   

In [ ]:
def train(model):
  num_batches = n_train // BATCH_SIZE
  model.train()
  t_loss, t_met, proc = 0, torch.tensor([0., 0., 0., 0.]), 0
  with tqdm(total=num_batches) as pbar:
    pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
    for batch in range(num_batches):
      X = torch.load(f"./Preproc/train_X_{batch}.torch")
      y = torch.load(f"./Preproc/train_y_{batch}.torch")
      loss, met = model.fit(X, y)
      t_loss += loss*len(X)
      t_met += met
      proc += len(X)
      pbar.update(1)
      pbar.set_description(f"Avg. Loss: {t_loss/proc: .4f}, Avg. Accuracy: {(t_met[0] + t_met[3])/proc: .4f}")
  return (t_loss/n_train, t_met/n_train)

def test(model):
  num_batches = n_val // BATCH_SIZE
  model.eval()
  t_loss, t_met, proc = 0, torch.tensor([0., 0., 0., 0.]), 0
  with tqdm(total=num_batches) as pbar:
    pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
    with torch.no_grad():
      for batch in range(num_batches):
        X = torch.load(f"./Preproc/val_X_{batch}.torch")
        y = torch.load(f"./Preproc/val_y_{batch}.torch")
        loss, met = model.test(X, y)
        t_loss += loss*len(X)
        t_met += met
        proc += len(X)
        pbar.update(1)
        pbar.set_description(f"Avg. Loss: {t_loss/proc : .4f}, Avg. Accuracy: {(t_met[0] + t_met[3])/proc: .4f}")
  return (t_loss/n_val, t_met/n_val)

In [ ]:
unet = DoubleUNet().to(DEVICE)

In [ ]:
A = unet(torch.randn((1, 3, IM_H, IM_W)).to(DEVICE))

In [ ]:
A[1].shape

In [ ]:
for t in range(EPOCHS):
  torch.cuda.empty_cache()
  print(f"Epoch {t+1} ---------------------")
  print(f"Training Set -----")
  train_loss, _ = train(unet)
  print(f"Validation Set -----")
  train_loss, met = test(unet)
  print(f"\rIoU: {met[0] / (met[0] + met[1] + met[2]): .4f}, Dice: {met[0] / (met[0] + 0.5 * (met[1] + met[2])): .4f}, Acc: {met[0] + met[3] : .4f}")